In [51]:
from math import log

#TODO: Remove punctuations.

positive_cls = 1
negative_cls = 0

def add_to_vocabulary(doc, vocab):
    words = doc.split(' ')
    
    for word in words:
        if word not in vocab:
            vocab[word] = 1
        else:
            vocab[word] += 1
            
def find_log_likelyhood(word, cls_vocab, cls_den):
    if word in cls_vocab:
        n = cls_vocab[word]
    else:
        n = 0
        
    return log((n + 1)/cls_den)

def calc_denominator(vocab, cls):
    s = 0
    for word in vocab['all']:
        if word in vocab[cls]:
            s += vocab[cls][word] + 1
    
    return s

def train_naive_bayes_classifier(filename):
    with open(filename) as f:
        n_positive_docs = 0
        n_negative_docs = 0
        log_likelyhood = {positive_cls: {}, negative_cls: {}}
        
        # a dictionary of dictionaries to hold all vocabulary
        vocabulary = {'all': {}, positive_cls: {}, negative_cls: {}}
        
        for line in f:
            record = line.rstrip().split('\t')
            text = record[0]
            cls = int(record[1])
            
            add_to_vocabulary(text, vocabulary['all'])
            
            if cls == negative_cls:
                n_negative_docs += 1
                add_to_vocabulary(text, vocabulary[negative_cls])
            elif cls == positive_cls:
                n_positive_docs += 1
                add_to_vocabulary(text, vocabulary[positive_cls])
                
        n_docs = n_positive_docs + n_negative_docs
        
        logprior = {}
        logprior[negative_cls] = log(n_negative_docs/n_docs)
        logprior[positive_cls] = log(n_positive_docs/n_docs)
                
        positive_den = calc_denominator(vocabulary, positive_cls)
        negative_den = calc_denominator(vocabulary, negative_cls)
        
        for word in vocabulary['all']:
            log_likelyhood[positive_cls][word] = find_log_likelyhood(word, vocabulary[positive_cls], positive_den)
            log_likelyhood[negative_cls][word] = find_log_likelyhood(word, vocabulary[negative_cls], negative_den)
            
    print('Number of positive docs: ', n_positive_docs)
    print('Number of negative docs', n_negative_docs)
    print('Total number of docs', n_docs)
    
    return log_likelyhood, logprior, vocabulary

def argmax(sum_dict):
    if sum_dict[positive_cls] > sum_dict[negative_cls]:
        return positive_cls
    else:
        return negative_cls

# the doc should be a list of words
def classify_doc(log_likelyhood, logprior, vocabulary, doc):
    classes = [positive_cls, negative_cls]
    s = {}
    s[positive_cls] = logprior[positive_cls]
    s[negative_cls] = logprior[negative_cls]
    
    for cls in classes:
        for word in doc:
            if word in vocabulary['all']:
                s[cls] += log_likelyhood[cls][word]
    return argmax(s)

log_likelyhood, logprior, vocabulary = train_naive_bayes_classifier('data/imdb_labelled.txt')
classify_doc(log_likelyhood, logprior, vocabulary, 'This is good product'.split(' '))


Number of positive docs:  500
Number of negative docs 500
Total number of docs 1000


1